### Importación de librerías

In [1]:
import cv2
import os
import mediapipe as mp
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Definición de variables

In [2]:
DATASET_PATH = "../../dataset/gestos_cuerpo"
MODEL_FILE = "modelo_gestos_mano.pkl"

### Lógica de entreno

In [3]:
# Configuración de MediaPipe para IMÁGENES ESTÁTICAS
mp_hands = mp.solutions.hands
# Importante: static_image_mode=True hace que sea más preciso para fotos sueltas
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

data = []
labels = []

# Mapeo de carpetas a etiquetas numéricas
mapa_etiquetas = {
    "0_Neutro": 0,
    "1_Mano_Der_Arriba": 1,
    "2_Mano_Izq_Arriba": 2,
    "3_Punos_Cerrados": 3,
    "4_Pulgar_Arriba": 4,
    "5_Victoria": 5,
    "6_Rock": 6,
    "7_Llamada": 7,
    "8_Ok": 8
}

print("--- INICIANDO PROCESAMIENTO DE FOTOS ---")

# Recorrer cada carpeta del dataset
for nombre_carpeta, etiqueta_num in mapa_etiquetas.items():
    ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
    
    if not os.path.exists(ruta_carpeta):
        print(f"Advertencia: La carpeta {ruta_carpeta} no existe. Saltando...")
        continue
        
    archivos = os.listdir(ruta_carpeta)
    print(f"Procesando '{nombre_carpeta}': {len(archivos)} imágenes encontradas.")
    
    count_ok = 0
    count_fail = 0
    
    for archivo in archivos:
        if not (archivo.endswith(".jpg") or archivo.endswith(".png")):
            continue
            
        ruta_img = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(ruta_img)
        
        if img is None:
            continue

        # Convertir a RGB para MediaPipe
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extraer coordenadas (x, y) de los 21 puntos
                landmark_list = []
                for lm in hand_landmarks.landmark:
                    landmark_list.append(lm.x)
                    landmark_list.append(lm.y)
                
                data.append(landmark_list)
                labels.append(etiqueta_num)
                count_ok += 1
        else:
            # Si en alguna foto no sale la mano o está borrosa, la ignoramos
            count_fail += 1

    print(f"  -> {count_ok} procesadas correctamente | {count_fail} fallidas (no se detectó mano)")

# --- ENTRENAMIENTO ---
if len(data) > 0:
    print("\n--- ENTRENANDO MODELO ---")
    data = np.array(data)
    labels = np.array(labels)

    # (Opcional) Separar un 20% para testear qué tan bueno es el modelo
    x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

    model = RandomForestClassifier()
    model.fit(x_train, y_train)

    # Validar precisión
    y_pred = model.predict(x_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión del modelo: {accuracy * 100:.2f}%")

    # Guardar modelo final
    model.fit(data, labels) # Re-entrenamos con todos los datos para el archivo final
    
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    
    print(f"¡ÉXITO! Modelo guardado como '{MODEL_FILE}'.")
    print("Ahora puedes ejecutar 'simon_game.py'")

else:
    print("ERROR: No se pudieron extraer datos. Revisa tus fotos o la iluminación.")

--- INICIANDO PROCESAMIENTO DE FOTOS ---
Procesando '0_Neutro': 30 imágenes encontradas.


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


  -> 15 procesadas correctamente | 14 fallidas (no se detectó mano)
Procesando '1_Mano_Der_Arriba': 303 imágenes encontradas.
  -> 291 procesadas correctamente | 12 fallidas (no se detectó mano)
Procesando '2_Mano_Izq_Arriba': 303 imágenes encontradas.
  -> 292 procesadas correctamente | 11 fallidas (no se detectó mano)
Procesando '3_Punos_Cerrados': 323 imágenes encontradas.
  -> 291 procesadas correctamente | 32 fallidas (no se detectó mano)
Procesando '4_Pulgar_Arriba': 300 imágenes encontradas.
  -> 271 procesadas correctamente | 29 fallidas (no se detectó mano)
Procesando '5_Victoria': 300 imágenes encontradas.
  -> 256 procesadas correctamente | 44 fallidas (no se detectó mano)
Procesando '6_Rock': 300 imágenes encontradas.
  -> 261 procesadas correctamente | 39 fallidas (no se detectó mano)
Procesando '7_Llamada': 300 imágenes encontradas.
  -> 280 procesadas correctamente | 20 fallidas (no se detectó mano)
Procesando '8_Ok': 300 imágenes encontradas.
  -> 282 procesadas correct

In [ ]:
import os
import cv2
import numpy as np
import pickle
import mediapipe as mp
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score

# --- CONFIGURACIÓN ---
# Ajusta esta ruta si tu carpeta está en otro lugar
DATASET_PATH = "../../dataset/gestos_cuerpo" 
MODEL_FILE = "modelo_body_red_augmentation.pkl"

# --- MEDIA PIPE HANDS ---
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,       # Detectar solo 1 mano por foto para simplificar el modelo
    min_detection_confidence=0.5
)

# --- ETIQUETAS ACTUALIZADAS SEGÚN TU IMAGEN ---
mapa_etiquetas = {
    "0_Neutro": 0,
    "1_Mano_Der_Arriba": 1,
    "2_Mano_Izq_Arriba": 2,
    "3_Punos_Cerrados": 3,
    "4_Pulgar_Arriba": 4,
    "5_Victoria": 5,
    "6_Rock": 6,
    "7_Llamada": 7,
    "8_Ok": 8
}

# --- FUNCIÓN DE NORMALIZACIÓN ---
def normalizar_puntos(landmarks):
    # Convertir landmarks a numpy array (x, y) ignorando z
    coords = np.array([[lm.x, lm.y] for lm in landmarks])
    
    # 1. Centrar en el origen (restar el centroide)
    centroid = np.mean(coords, axis=0)
    centered = coords - centroid
    
    # 2. Escalar (dividir por la distancia máxima absoluta)
    max_dist = np.max(np.abs(centered))
    
    if max_dist > 0:
        normalized = centered / max_dist
    else:
        normalized = centered
        
    return normalized.flatten()

# --- DATA AUGMENTATION ---
def aumentar_datos(vector_original):
    variaciones = []
    variaciones.append(vector_original)
    
    puntos_2d = vector_original.reshape(-1, 2)

    # 1. Ruido ligero
    ruido = np.random.normal(0, 0.01, puntos_2d.shape)
    variaciones.append((puntos_2d + ruido).flatten())

    # 2. Escala (Zoom in/out leve)
    factor_escala = np.random.uniform(0.95, 1.05)
    variaciones.append((puntos_2d * factor_escala).flatten())
    
    # 3. Rotación (+/- 15 grados)
    theta = np.radians(np.random.uniform(-15, 15))
    c, s = np.cos(theta), np.sin(theta)
    matriz_rotacion = np.array(((c, -s), (s, c)))
    puntos_rotados = np.dot(puntos_2d, matriz_rotacion)
    variaciones.append(puntos_rotados.flatten())

    return variaciones

# --- PROCESAMIENTO DE IMÁGENES ---
data = []
labels = []
count_total = 0
count_ok = 0

print("--- INICIANDO PROCESAMIENTO DE IMÁGENES ---")
print(f"Buscando dataset en: {DATASET_PATH}")

for nombre_carpeta, etiqueta_num in mapa_etiquetas.items():
    ruta_carpeta = os.path.join(DATASET_PATH, nombre_carpeta)
    
    if not os.path.exists(ruta_carpeta):
        print(f"ADVERTENCIA: No se encontró la carpeta: {ruta_carpeta}")
        continue
    
    archivos = os.listdir(ruta_carpeta)
    print(f"Procesando clase '{nombre_carpeta}' ({len(archivos)} archivos)...")
    
    for archivo in archivos:
        if not (archivo.lower().endswith(('.jpg', '.png', '.jpeg'))): continue
        
        ruta_img = os.path.join(ruta_carpeta, archivo)
        img = cv2.imread(ruta_img)
        if img is None: continue
        
        # Procesar con MediaPipe Hands 
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = hands.process(img_rgb)
        
        if results.multi_hand_landmarks:
            count_ok += 1
            # Tomamos la primera mano detectada
            hand_landmarks = results.multi_hand_landmarks[0]
            
            # Normalizar
            vector_base = normalizar_puntos(hand_landmarks.landmark)
            
            # Data Augmentation (Generar variaciones para robustez)
            vectores_aumentados = aumentar_datos(vector_base)
            
            for v in vectores_aumentados:
                data.append(v)
                labels.append(etiqueta_num)
        
        count_total += 1

print(f"\nResumen: {count_ok} manos detectadas correctamente de {count_total} imágenes leídas.")

# --- ENTRENAMIENTO DEL MODELO ---
data = np.array(data)
labels = np.array(labels)

if len(data) > 0:
    print(f"\nIniciando entrenamiento con {len(data)} muestras (incluyendo augmentation)...")
    
    # Configuración del Perceptrón Multicapa (Red Neuronal)
    model = MLPClassifier(
        hidden_layer_sizes=(64, 32), # Capas ocultas
        activation='relu', 
        solver='adam', 
        max_iter=1500,
        random_state=42
    )

    # Validación cruzada para verificar calidad
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = cross_val_score(model, data, labels, cv=kfold)
    
    print(f"Precisión Media estimada (Cross-Validation): {scores.mean() * 100:.2f}%")

    # Entrenar modelo final con todos los datos
    model.fit(data, labels)
    
    # Guardar modelo
    with open(MODEL_FILE, 'wb') as f:
        pickle.dump(model, f)
    print(f"¡Éxito! Modelo guardado en '{MODEL_FILE}'")
    
else:
    print("\nERROR: No se generaron datos de entrenamiento. Revisa las rutas de las imágenes.")

--- INICIANDO PROCESAMIENTO DE IMÁGENES ---
Buscando dataset en: ../../dataset/gestos_cuerpo
Procesando clase '0_Neutro' (30 archivos)...


c:\Users\lucia\anaconda33\envs\memo_env\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Procesando clase '1_Mano_Der_Arriba' (303 archivos)...
Procesando clase '2_Mano_Izq_Arriba' (303 archivos)...
Procesando clase '3_Punos_Cerrados' (323 archivos)...
Procesando clase '4_Pulgar_Arriba' (300 archivos)...
Procesando clase '5_Victoria' (300 archivos)...
Procesando clase '6_Rock' (300 archivos)...
Procesando clase '7_Llamada' (300 archivos)...
Procesando clase '8_Ok' (300 archivos)...

Resumen: 2239 manos detectadas correctamente de 2458 imágenes leídas.

Iniciando entrenamiento con 8956 muestras (incluyendo augmentation)...
Precisión Media estimada (Cross-Validation): 99.20%
¡Éxito! Modelo guardado en 'modelo_gestos_cuerpo.pkl'
